<a href="https://colab.research.google.com/github/jdmartinev/ArtificialIntelligenceIM/blob/main/Lecture09/notebooks/L09_CNN_Transfer_pytorch_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de Aprendizaje en el Conjunto de Datos Caltech101

En este cuaderno, consideraremos un conjunto de datos más complejo que MNIST o CIFAR10. Las imágenes en Caltech101 son imágenes RGB (3 canales) con tamaño variable. Hay 101 clases diferentes. Intentaremos una práctica muy común en visión por computadora en la actualidad: transferencia de aprendizaje desde un modelo preentrenado en ImageNet.

## Plan de Trabajo:
- Modificar la red del ejercicio anterior (CIFAR-10) para trabajar con imágenes de 224x224.
- Entrenar el modelo por un tiempo en Caltech101 y ver qué tan lejos podemos llegar.
- Tomar un ResNet34 que fue preentrenado en ImageNet-1k y ajustarlo a Caltech101.
  - Considerar tanto entrenar solo la cabeza (el clasificador lineal al final de la red) como toda la red.
  - Deberíamos poder alcanzar un mejor rendimiento que nuestra red original en menos pasos de entrenamiento.
- Opcional: experimentar con otros modelos preentrenados de `timm` (ver información [aquí](https://github.com/rwightman/pytorch-image-models)).

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## Preliminares

In [ ]:
%matplotlib inline
from typing import List, Optional, Callable, Iterator
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torchvision.models as models

from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset, Subset
from datasets import load_dataset
from collections import Counter

sns.set_style("whitegrid")

def show_image(img, title=None):
    img = img.detach().cpu()
    img = img.permute((1, 2, 0)).numpy()
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean   # unnormalize
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.gca().tick_params(axis="both", which="both", bottom=False, left=False, labelbottom=False, labelleft=False)
    if title is not None:
        plt.title(title)

## Configuración de la base de datos

In [ ]:
# --- ImageNet normalization
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]
default_imagenet_normalization = transforms.Normalize(mean=imagenet_mean, std=imagenet_std)

# --- Transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.25, 1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    default_imagenet_normalization,
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    default_imagenet_normalization,
])

# Step 1: Load dataset
hf_dataset = load_dataset("flwrlabs/caltech101", split="train")  # all in one split

# Step 2: Count class occurrences
label_counts = Counter(hf_dataset['label'])

# Step 3: Keep only classes with ≤100 samples
keep_classes = sorted([label for label, count in label_counts.items() if count <= 100])

# Step 4: Subsample max 100 samples per class
indices = list(range(len(hf_dataset)))
random.shuffle(indices)

new_indices = []
class_counts = {i: 0 for i in keep_classes}

for i in indices:
    y = hf_dataset[i]['label']
    if y in keep_classes and class_counts[y] < 100:
        new_indices.append(i)
        class_counts[y] += 1

# Step 5: Translate labels
def translate_label(y, keep_classes):
    try:
        return keep_classes.index(y)
    except ValueError:
        return -1

labels = [translate_label(hf_dataset[i]['label'], keep_classes) for i in new_indices]
assert max(labels) == len(keep_classes) - 1

labels = np.array(labels)
# Step 6: Train-test split
test_size = 640
val_size = 640
train_size = len(new_indices) - test_size - val_size

# First, split into train+val and test
train_val_idx, test_idx = train_test_split(
    new_indices,
    test_size=test_size,
    stratify=labels,
    random_state=42
)

# Map from full dataset index to position in new_indices
idx_map = {idx: i for i, idx in enumerate(new_indices)}

# Get label indices for train_val
train_val_labels = np.array([labels[idx_map[i]] for i in train_val_idx])

# Then, split train+val into train and val
train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=val_size,
    stratify=train_val_labels,
    random_state=42
)

# --- Torch Dataset wrapper
class HFCaltech101(Dataset):
    def __init__(self, hf_dataset, indices, transform, keep_classes):
        self.hf_dataset = hf_dataset
        self.indices = indices
        self.transform = transform
        self.keep_classes = keep_classes

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        item = self.hf_dataset[self.indices[idx]]
        image = item['image'].convert('RGB')
        label = translate_label(item['label'], self.keep_classes)

        if self.transform:
            image = self.transform(image)
        return image, label

# --- Datasets and loaders
train_dataset = HFCaltech101(hf_dataset, train_idx, train_transform, keep_classes)
val_dataset = HFCaltech101(hf_dataset, val_idx, test_transform, keep_classes)
test_dataset = HFCaltech101(hf_dataset, test_idx, test_transform, keep_classes)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


### Un Vistazo Más Cercano al Conjunto de Datos

Primero, trazamos el tamaño de cada clase y observamos que la distribución de clases no es uniforme.

Luego, mostramos ejemplos aleatorios del conjunto de datos, anotados con la etiqueta de la clase y el índice.
Ten en cuenta que las imágenes de entrenamiento incluyen aumentos estándar que se utilizan típicamente en modelos de visión (definidos anteriormente).


In [ ]:
full_label_names = hf_dataset.features['label'].names
categories = [full_label_names[i] for i in keep_classes]

label_idxs, counts = np.unique(labels, return_counts=True)
plt.figure(figsize=(20, 4.2))
sns.barplot(
    x=[categories[label] for label in label_idxs],
    y=counts,
    color=sns.color_palette('muted')[0]
)
plt.xticks(rotation=90)
plt.xlabel("Class")
plt.ylabel("Number of examples")
plt.title("Class distribution")
plt.tight_layout()
plt.show()


def show_dataset_examples(dataloader):
    images, labels = next(iter(dataloader))
    with sns.axes_style("white"):
        fig, axes = plt.subplots(4, 8, figsize=(16, 9.5))
    axes = [ax for axes_ in axes for ax in axes_]   # flatten

    for j, (img, label) in enumerate(zip(images[:32], labels[:32])):
        plt.sca(axes[j])
        show_image(img, title=f"{categories[label.item()]} ({label.item()})")

    plt.tight_layout()
    plt.show()


print("\n\nTrain images (including augmentations):")
show_dataset_examples(train_loader)
print("\n\nTrain images (including augmentations):")
show_dataset_examples(val_loader)
print("\n\nTest images:")
show_dataset_examples(test_loader)

## Definir una Red Neuronal

**Asignación 1:** Adapta la CNN del laboratorio anterior (CIFAR-10) para manejar imágenes de 224x224. Recomendamos reducir significativamente el tamaño de los tensores antes de aplanarlos, añadiendo capas convolucionales con `stride > 1` o capas de [MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) (pero también puedes considerar [AvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html)).


In [ ]:
class Model(nn.Module):

    def __init__(self, num_classes):
        super().__init__()

        self.net = ...   # Your code here!

    def forward(self, x):
        return self.net(x)


model = Model(num_classes=len(np.unique(labels)))
device = torch.device('cuda')  # use cuda or cpu
model = model.to(device)
print(model)

Model()


## Definir la Función de Pérdida y el Optimizador

**Asignación 2:** Implementa el criterio y el optimizador, como en el cuaderno anterior.

In [ ]:
loss_fn = None  # Your code here!
optimizer = None  # Your code here!

## Entrenar la red

In [ ]:
# Test the forward pass with dummy data
out = model(torch.randn(2, 3, 224, 224).to(device))
print("Output shape:", out.size())

In [ ]:
def accuracy(targets, predictions):
    return (predictions == targets).float().mean().item()

num_epochs = 10
validation_every_steps = 50

step = 0
model.train()

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    train_accuracies_batches = []
    train_loss_accum = 0
    train_loss_steps = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # --- Forward pass
        output = model(inputs)

        # --- Loss
        loss = loss_fn(output, targets)
        train_losses.append(loss.item())
        train_loss_accum += loss.item()
        train_loss_steps += 1

        # --- Backprop and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # --- Accuracy
        predictions = output.argmax(dim=1)
        train_accuracies_batches.append(accuracy(targets, predictions))

        step += 1

        # --- Validation
        if step % validation_every_steps == 0:
            avg_train_loss = train_loss_accum / train_loss_steps
            avg_train_acc = np.mean(train_accuracies_batches)
            train_accuracies.append(avg_train_acc)

            train_loss_accum = 0
            train_loss_steps = 0
            train_accuracies_batches = []

            model.eval()
            val_accuracies_batches = []
            val_loss_accum = 0
            num_val_samples = 0

            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    output = model(inputs)

                    val_loss = loss_fn(output, targets)
                    val_loss_accum += val_loss.item() * len(inputs)
                    num_val_samples += len(inputs)

                    predictions = output.argmax(dim=1)
                    val_accuracies_batches.append(accuracy(targets, predictions) * len(inputs))

            val_accuracy = np.sum(val_accuracies_batches) / num_val_samples
            avg_val_loss = val_loss_accum / num_val_samples
            val_accuracies.append(val_accuracy)
            val_losses.append(avg_val_loss)

            print(f"Step {step:<5}   "
                  f"train loss: {avg_train_loss:.4f}   train acc: {avg_train_acc:.4f}   "
                  f"val loss: {avg_val_loss:.4f}   val acc: {val_accuracy:.4f}")

            model.train()

print("Finished training.")

# Evaluar la red en el conjunto de test

In [ ]:
with torch.no_grad():
    model.eval()
    test_accuracies = []
    num_samples = 0

    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        output = model(inputs)
        predictions = output.argmax(dim=1)

        # Accumulate correct predictions × batch size
        batch_acc = accuracy(targets, predictions) * len(inputs)
        test_accuracies.append(batch_acc)
        num_samples += len(inputs)

    test_accuracy = np.sum(test_accuracies) / num_samples
    print(f"Test accuracy: {test_accuracy:.3f}")

## Usando un Modelo Preentrenado

Aquí cargaremos un ResNet34 que fue preentrenado en ImageNet. Luego, descartaremos el clasificador lineal al final de la red (la "cabeza" de la red) y lo reemplazaremos con uno nuevo que produzca el número deseado de logits para la clasificación. Para obtener una idea general de la estructura del modelo, lo imprimimos a continuación.

El argumento `finetune_entire_model` en `initialize_model()` controla si se ajusta toda la red preentrenada. Cuando esto es `False`, solo se entrena la cabeza lineal y el resto del modelo se mantiene fijo. La idea es que las características extraídas por el modelo de ImageNet, hasta la capa de clasificación final, también son muy informativas en otros conjuntos de datos (ver, por ejemplo, [este artículo](https://arxiv.org/abs/1910.04867) sobre la transferibilidad de representaciones profundas en modelos de visión grandes).

Comenzaremos aquí entrenando solo la cabeza lineal. Puedes experimentar con diferentes modelos y variaciones.

A continuación, definimos el modelo y descartamos el que acabamos de entrenar. Después de eso, puedes volver a la sección "Definir la Función de Pérdida y el Optimizador" y volver a ejecutar el cuaderno desde allí, para entrenar y evaluar el nuevo modelo.


In [ ]:
def initialize_model(model_name: str, *, num_classes: int, finetune_entire_model: bool = False):
    """Initializes a torchvision model with a new classifier and optionally freezes the backbone.

    Returns:
        model: the PyTorch model
        info: a dict with total and trainable parameter counts
    """

    print(
        f"Loading torchvision model '{model_name}', with "
        f"finetune_entire_model={finetune_entire_model}, changing the "
        f"last layer to output {num_classes} logits."
    )

    # Supported torchvision models (you can extend this list)
    available_models = {
        "resnet18": models.resnet18,
        "resnet34": models.resnet34,
        "resnet50": models.resnet50,
        "mobilenet_v2": models.mobilenet_v2,
        "efficientnet_b0": models.efficientnet_b0,
        "densenet121": models.densenet121,
    }

    if model_name not in available_models:
        raise ValueError(f"Model '{model_name}' is not supported.")

    # Load the model with pretrained weights
    model = available_models[model_name](weights="DEFAULT")

    # Replace final classifier
    if "resnet" in model_name or "resnext" in model_name:
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
        classifier_layer = model.fc
    elif "mobilenet" in model_name:
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)
        classifier_layer = model.classifier[1]
    elif "efficientnet" in model_name:
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)
        classifier_layer = model.classifier[1]
    elif "densenet" in model_name:
        in_features = model.classifier.in_features
        model.classifier = nn.Linear(in_features, num_classes)
        classifier_layer = model.classifier
    else:
        raise ValueError(f"Model structure not recognized for '{model_name}'.")

    # Optionally freeze all layers except the final classifier
    if not finetune_entire_model:
        for param in model.parameters():
            param.requires_grad = False
        classifier_layer.weight.requires_grad_()
        classifier_layer.bias.requires_grad_()

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    return model, {
        "num_model_parameters": total_params,
        "num_trainable_model_parameters": trainable_params,
    }

model, data = initialize_model('resnet34', num_classes=len(np.unique(labels)), finetune_entire_model=False)

print(model)
print("Number of model parameters:", data["num_model_parameters"])
print("Number of trainable parameters:", data["num_trainable_model_parameters"])

device = torch.device('cuda')  # use cuda or cpu
model = model.to(device)

**Asignación 3:**

1. Entrena el clasificador lineal sobre la red preentrenada y observa qué tan rápido puedes obtener buenos resultados, en comparación con entrenar una red más pequeña desde cero como hicimos anteriormente.

2. Vuelve atrás y cambia el argumento para ajustar toda la red (`finetune_entire_model`), tal vez ajusta la tasa de aprendizaje, y observa si puedes obtener un mejor rendimiento que antes y si encuentras algún problema.

3. Opcional: experimenta con `timm`: prueba modelos más pequeños o más grandes, incluyendo modelos de última generación, por ejemplo, basados en transformadores de visión (ViT) o MLP-Mixers.

4. Describe brevemente lo que hiciste y cualquier experimento que realizaste, así como los resultados que obtuviste.
¿Algo te sorprendió durante el ejercicio?

5. Escribe las lecciones o ideas clave que obtuviste durante este ejercicio.
